In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd

C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:455: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:456: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:457: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

Load training data 

In [2]:
training_data_df = pd.read_csv("./Datasets/sales_data_training.csv", dtype=float)

In [3]:
training_data_df.head()

,critic_rating,is_action,is_exclusive_to_us,is_portable,is_role_playing,is_sequel,is_sports,suitable_for_kids,total_earnings,unit_price
0,3.5,1.0,0.0,1.0,0.0,1.0,0.0,0.0,132717.0,59.99
1,4.5,0.0,0.0,0.0,0.0,1.0,1.0,0.0,83407.0,49.99
2,3.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,62423.0,49.99
3,4.5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,69889.0,39.99
4,4.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,161382.0,59.99


Prepare X and y

In [4]:
X_training = training_data_df.drop('total_earnings', axis=1).values
Y_training = training_data_df[['total_earnings']].values

Load test data 

In [5]:
test_data_df = pd.read_csv("./Datasets/sales_data_test.csv", dtype=float)

In [6]:
test_data_df.head()

,critic_rating,is_action,is_exclusive_to_us,is_portable,is_role_playing,is_sequel,is_sports,suitable_for_kids,total_earnings,unit_price
0,3.5,1.0,1.0,1.0,0.0,1.0,0.0,1.0,247537.0,59.99
1,2.5,0.0,0.0,0.0,1.0,1.0,0.0,0.0,73960.0,59.99
2,3.5,0.0,0.0,0.0,0.0,1.0,1.0,0.0,82671.0,59.99
3,4.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,137456.0,39.99
4,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,89639.0,59.99


In [7]:
X_testing = test_data_df.drop('total_earnings', axis=1).values
Y_testing = test_data_df[['total_earnings']].values

### Data Preprocessing 

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
X_scaler = MinMaxScaler(feature_range=(0, 1))
Y_scaler = MinMaxScaler(feature_range=(0, 1))

scale training data 

In [10]:
X_scaled_training = X_scaler.fit_transform(X_training)
Y_scaled_training = Y_scaler.fit_transform(Y_training)

scale test data 

In [11]:
X_scaled_testing = X_scaler.transform(X_testing)
Y_scaled_testing = Y_scaler.transform(Y_testing)

### Neural Netwok configuration settings 

Model Parameters 

In [12]:
learning_rate = 0.001
training_epochs = 100

Define how many inputs and outputs

In [13]:
number_of_inputs = 9
number_of_outputs = 1

Define how many neurons we want in each layer

In [14]:
layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

### Building Computation Graph

Input layer 

In [15]:
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, number_of_inputs))

Layer 1

In [16]:
with tf.variable_scope('layer_1'):
    
    weights = tf.get_variable(name="weights1", shape=[number_of_inputs, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)


Layer 2

In [17]:
with tf.variable_scope('layer_2'):
    weights = tf.get_variable(name="weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

Layer 3

In [18]:

with tf.variable_scope('layer_3'):
    weights = tf.get_variable(name="weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)


Output Layer

In [19]:
with tf.variable_scope('output'):
    weights = tf.get_variable(name="weights4", shape=[layer_3_nodes, number_of_outputs], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases4", shape=[number_of_outputs], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output, weights) + biases

Cost Function 

In [20]:
with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

Optimizer function

In [21]:
with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### Training Phase

In [22]:
init = tf.global_variables_initializer()

In [23]:
session= tf.Session()

In [24]:
session.run(init)

In [25]:
for epoch in range(training_epochs):
    
    session.run(optimizer, feed_dict={X: X_scaled_training, Y: Y_scaled_training})

# Scales Prediction 

In [26]:
Y_predicted_scaled = session.run(prediction, feed_dict={X: X_scaled_testing})

In [27]:
Y_predicted = Y_scaler.inverse_transform(Y_predicted_scaled)

In [28]:
real_earnings = test_data_df['total_earnings'].values[0]
predicted_earnings = Y_predicted[0][0]

In [29]:
print("The actual earnings of Game #1 were ${}".format(real_earnings))
print("Our neural network predicted earnings of ${}".format(predicted_earnings))


The actual earnings of Game #1 were $247537.0
Our neural network predicted earnings of $244670.40625


# Save the model for the future use 

In [30]:
saver = tf.train.Saver()

In [31]:
save_path = saver.save(session, "logs/trained_model.ckpt")
print("Model saved: {}".format(save_path))

Model saved: logs/trained_model.ckpt


In [32]:
session.close()